# Full Configuration Interaction (FCI)

This notebook implements a small-scale FCI solver assuming you provide **MO-basis** one- and two-electron integrals. It also explains how to obtain those integrals from a Hartree–Fock (SCF) calculation.

## 1) Modeling de wave function as a linear combination of terms

**Inputs required (in MO basis, spatial orbitals):**
- One-electron integrals $h[p,q] = <p | T + V_ne | q>$ (K×K)
- Two-electron Coulomb integrals in physicist's notation $(pq|rs)$ (K×K×K×K)

**Workflow to obtain them:**
1. Run RHF/SCF in your favorite code (PySCF, Psi4, etc.) to get MO coefficients $C$ and AO integrals.
2. Transform AO->MO:
   - $h_mo = C.T @ h_ao @ C$
   - $(pq|rs)_mo = sum_mu,nu,la,si C_mu,p C_nu,q C_la,r C_si,s (mu nu | la si)_ao$

**Reference snippet with PySCF (for guidance):**
$$$python
from pyscf import gto, scf, ao2mo
mol = gto.M(atom='H 0 0 0; H 0 0 0.74', basis='sto-3g')
mf = scf.RHF(mol).run()
C = mf.mo_coeff               # (nAO, K)
h_ao = mf.get_hcore()         # AO 1e integrals
h_mo = C.T @ h_ao @ C         # MO 1e integrals (K,K)
eri_ao = ao2mo.kernel(mol, C)            # compressed MO ERIs
eri_mo = ao2mo.restore(1, eri_ao, C.shape[1])  # full (K,K,K,K), physicist's notation
$$$
Use $h_mo$ and $eri_mo$ as inputs in the cells below.

## 2) Utilities: determinant bitstrings and combinatorics
We represent a determinant by two integers: **alpha bits** and **beta bits**, each bit indicates occupation of a spatial orbital.

In [ ]:
import itertools, math, numpy as np

def bits_set_positions(x):
    out, pos = [], 0
    while x:
        if x & 1:
            out.append(pos)
        x >>= 1; pos += 1
    return out

def combinations_of_bits(K, n):
    for comb in itertools.combinations(range(K), n):
        x = 0
        for i in comb:
            x |= (1 << i)
        yield x

def count_determinants(K, Nalpha, Nbeta):
    return math.comb(K, Nalpha) * math.comb(K, Nbeta)

print('Example N_det (K=6, 2α,2β):', count_determinants(6,2,2))

## 3) Slater–Condon rules (RHF spatial-orbital integrals)

**Diagonal** for determinant with occupied sets Oα and Oβ:
$H_DD = sum_{i in Oα} h[ii] + sum_{j in Oβ} h[jj]$
+ $1/2 * sum_{i,j in Oα} [(ii|jj) - (ij|ji)] + 1/2 * sum_{i,j in Oβ} [(ii|jj) - (ij|ji)]$
+ $sum_{i in Oα, j in Oβ} (ii|jj)$ (Coulomb only across spins).

**Single excitation α** i→a:
$h[a,i] + sum_{j in Oα, j!=i} [(a j | i j) - (a j | j i)] + sum_{j in Oβ} (a j | i j)$
(analogous for β).

**Double excitations:** same-spin → $<ab||ij> = (a i | b j) - (a j | b i)$; mixed-spin → $(a i | b j)$.


In [ ]:
def excitation_phase(occ_list, i, a):
    occ = list(occ_list)
    assert i in occ and a not in occ
    idx_i = occ.index(i)
    occ_removed = occ[:idx_i] + occ[idx_i+1:]
    insert_pos = sum(1 for x in occ_removed if x < a)
    swaps = abs(insert_pos - idx_i)
    return -1 if (swaps % 2) else 1

def occ_lists(det):
    a_bits, b_bits = det
    return bits_set_positions(a_bits), bits_set_positions(b_bits)

def diagonal_energy(h, eri, det):
    Oa, Ob = occ_lists(det)
    E = 0.0
    for i in Oa: E += h[i,i]
    for j in Ob: E += h[j,j]
    for i in Oa:
        for j in Oa:
            E += 0.5 * (eri[i,i,j,j] - eri[i,j,j,i])
    for i in Ob:
        for j in Ob:
            E += 0.5 * (eri[i,i,j,j] - eri[i,j,j,i])
    for i in Oa:
        for j in Ob:
            E += eri[i,i,j,j]
    return E

def single_matrix_element_alpha(h, eri, Oa, Ob, i, a):
    val = h[a,i]
    for j in Oa:
        if j == i: continue
        val += (eri[a,j,i,j] - eri[a,j,j,i])
    for j in Ob:
        val += eri[a,j,i,j]
    return val

def single_matrix_element_beta(h, eri, Oa, Ob, j, b):
    val = h[b,j]
    for i in Ob:
        if i == j: continue
        val += (eri[b,i,j,i] - eri[b,i,i,j])
    for i in Oa:
        val += eri[b,i,j,i]
    return val

def double_same_spin(h, eri, i, j, a, b):
    return eri[a,i,b,j] - eri[a,j,b,i]

def double_mixed_spin(h, eri, i_alpha, a_alpha, j_beta, b_beta):
    return eri[a_alpha,i_alpha,b_beta,j_beta]


## 4) Build the FCI Hamiltonian (explicit, for small spaces)
This routine creates the full matrix $H$ by looping over determinant pairs and applying Slater–Condon rules.

In [ ]:
def diff_alpha_beta(detI, detJ):
    aI, bI = detI
    aJ, bJ = detJ
    OaI, ObI = bits_set_positions(aI), bits_set_positions(bI)
    OaJ, ObJ = bits_set_positions(aJ), bits_set_positions(bJ)

    lost_a = [x for x in OaI if x not in OaJ]
    gained_a = [x for x in OaJ if x not in OaI]
    lost_b = [x for x in ObI if x not in ObJ]
    gained_b = [x for x in ObJ if x not in ObI]

    diffs_a = list(zip(lost_a, gained_a))
    diffs_b = list(zip(lost_b, gained_b))
    total_ex = len(lost_a) + len(lost_b)
    return diffs_a, diffs_b, total_ex

def build_fci_hamiltonian(h, eri, K, Nalpha, Nbeta):
    alpha_dets = list(combinations_of_bits(K, Nalpha))
    beta_dets  = list(combinations_of_bits(K, Nbeta))
    dets = [(a,b) for a in alpha_dets for b in beta_dets]
    Ndet = len(dets)

    H = np.zeros((Ndet, Ndet), dtype=float)
    occA = [bits_set_positions(a) for a,_ in dets]
    occB = [bits_set_positions(b) for _,b in dets]

    for I, detI in enumerate(dets):
        OaI, ObI = occA[I], occB[I]
        H[I,I] = diagonal_energy(h, eri, detI)
        for J in range(I+1, Ndet):
            detJ = dets[J]
            diffa, diffb, total_ex = diff_alpha_beta(detI, detJ)
            if total_ex == 0 or total_ex > 2:
                continue
            val = 0.0
            phase = 1
            if total_ex == 1:
                if len(diffa) == 1 and len(diffb) == 0:
                    i,a = diffa[0]
                    phase *= excitation_phase(OaI, i, a)
                    val = single_matrix_element_alpha(h, eri, OaI, ObI, i, a)
                elif len(diffb) == 1 and len(diffa) == 0:
                    j,b = diffb[0]
                    phase *= excitation_phase(ObI, j, b)
                    val = single_matrix_element_beta(h, eri, OaI, ObI, j, b)
            elif total_ex == 2:
                if len(diffa) == 2 and len(diffb) == 0:
                    (i,a),(j,b) = diffa
                    phase *= excitation_phase(OaI, i, a)
                    Oa_after = [x for x in OaI if x!=i] + [a]
                    Oa_after.sort()
                    phase *= excitation_phase(Oa_after, j, b)
                    val = double_same_spin(h, eri, i, j, a, b)
                elif len(diffb) == 2 and len(diffa) == 0:
                    (j,b),(k,c) = diffb
                    phase *= excitation_phase(ObI, j, b)
                    Ob_after = [x for x in ObI if x!=j] + [b]
                    Ob_after.sort()
                    phase *= excitation_phase(Ob_after, k, c)
                    val = double_same_spin(h, eri, j, k, b, c)
                elif len(diffa) == 1 and len(diffb) == 1:
                    i,a = diffa[0]
                    j,b = diffb[0]
                    phase *= excitation_phase(OaI, i, a) * excitation_phase(ObI, j, b)
                    val = double_mixed_spin(h, eri, i, a, j, b)

            H[I,J] = phase * val
            H[J,I] = H[I,J]
    return H, dets


## 5) Demo with mock integrals
This creates a toy set of symmetric integrals (for **testing only**), builds $H$, and diagonalizes it.

In [ ]:
def mock_integrals(K, seed=0):
    rng = np.random.default_rng(seed)
    A = rng.normal(size=(K,K))
    h = (A + A.T) / 2.0
    eri = rng.normal(size=(K,K,K,K))
    eri = 0.5*(eri + eri.swapaxes(0,1))
    eri = 0.5*(eri + eri.swapaxes(2,3))
    eri = 0.5*(eri + eri.swapaxes(0,2).swapaxes(1,3))
    return h, eri

K, Nalpha, Nbeta = 4, 1, 1
h, eri = mock_integrals(K, seed=42)
H, dets = build_fci_hamiltonian(h, eri, K, Nalpha, Nbeta)
evals, evecs = np.linalg.eigh(H)
print('K, Nα, Nβ =', K, Nalpha, Nbeta)
print('N_det      =', len(dets))
print('E(min)     =', evals[0])

## 6) Counting how many terms in the CI sum
The number of determinants (and thus coefficients in the expansion) is $C(K, Nα) * C(K, Nβ)$.
Connectivity per determinant (spin-orbital counting): $N_s = N (M-N)$ and $N_d = C(N,2) * C(M-N,2)$.